In [24]:
#import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

LoadError: ArgumentError: Package JuliaAcademyData not found in current path:
- Run `import Pkg; Pkg.add("JuliaAcademyData")` to install the JuliaAcademyData package.


# Fast (serial) programming with Julia

Yes, this is a parallel computing course — but to write efficient parallel
programs we first must learn how to write fast serial Julia code. This is
a rapid primer in high performance (serial) programming.

I _highly_ recommend reviewing the [Performance Tips](https://docs.julialang.org/en/v1.1/manual/performance-tips/)
in the manual. This is only going to briefly introduce some of the main concepts.

## Measure, measure, measure.

It is very easy to experiment in Julia; you can rapidly try many options and
see what is the fastest.

Use the [BenchmarkTools](https://github.com/JuliaCI/BenchmarkTools.jl) package:

In [25]:
using BenchmarkTools

"""
    findclosest(data, point)

A simple example that returns the element in `data` that is closest to `point`
"""
function findclosest(data, point)
    _, index =  findmin(abs.(data .- point))
    return data[index]
end
data = rand(5000)
findclosest(data, 0.5)

0.4999157852134348

In [26]:
@time findclosest(data, 0.5)

  0.000022 seconds (3 allocations: 39.156 KiB)


0.4999157852134348

In [27]:
@benchmark findclosest($data, $0.5)

BenchmarkTools.Trial: 
  memory estimate:  39.14 KiB
  allocs estimate:  2
  --------------
  minimum time:     11.938 μs (0.00% GC)
  median time:      14.547 μs (0.00% GC)
  mean time:        15.215 μs (3.70% GC)
  maximum time:     774.624 μs (97.02% GC)
  --------------
  samples:          10000
  evals/sample:     1

### Profile!

In [28]:
using Profile

Profile.clear()
@profile for _ in 1:100000; findclosest(data, 0.5); end

Profile.print(maxdepth=11)

Overhead ╎ [+additional indent] Count File:Line; Function
   ╎1355 @Base/task.jl:356; (::IJulia.var"#15#18")()
   ╎ 1355 ...lia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
   ╎  1355 @Base/essentials.jl:709; invokelatest
   ╎   1355 @Base/essentials.jl:710; #invokelatest#1
   ╎    1355 .../execute_request.jl:67; execute_request(::ZMQ.Socket, ...
   ╎     1355 ...SoftGlobalScope.jl:65; softscope_include_string(::Mo...
   ╎    ╎ 1355 @Base/loading.jl:1091; include_string(::Function, ...
   ╎    ╎  1355 ...le/src/Profile.jl:28; top-level scope
  9╎    ╎   1354 In[28]:4; macro expansion
   ╎    ╎    1339 In[25]:9; findclosest(::Array{Float64...
   ╎    ╎     333  @Base/broadcast.jl:837; materialize
   ╎    ╎    ╎ 333  @Base/broadcast.jl:862; copy
   ╎    ╎     1006 @Base/reducedim.jl:838; findmin
   ╎    ╎    ╎ 1006 @Base/reducedim.jl:838; #findmin#652
  1╎    ╎    6    In[25]:10; findclosest(::Array{Float64...
  5╎    ╎     5    @Base/array.jl:809; getindex
   ╎    ╎   1    @Base/range.jl

### Iterate!

Before we had:
```julia
function findclosest(data, point)
    _, index =  findmin(abs.(data .- point))
    return data[index]
end
```

Let's come up with a new definition that can combine the two operations:

In [29]:
function findclosest2(data, point)
    bestval = first(data)
    bestdist = abs(bestval - point)
    for elt in data
        dist = abs(elt - point)
        if dist < bestdist
            bestval = elt
            bestdist = dist
        end
    end
    return bestval
end

# And do a spot-check to make sure we did the optimization correctly:
findclosest2(data, 0.5) == findclosest(data, .5)

true

In [30]:
@benchmark findclosest2($data, $0.5)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.186 μs (0.00% GC)
  median time:      6.697 μs (0.00% GC)
  mean time:        6.963 μs (0.00% GC)
  maximum time:     31.217 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     5

## A quick word on macros

Macros are those funny things starting with `@`. They can reinterpret what
you write and do something different — essentially introducing a new keyword.

For example, the `@assert` macro simply takes an expression and throws an
exception if it returns `false`.

In [31]:
@assert 2+2 == 4

It does this by literally re-writing what you wrote. You can see it in action
with `@macroexpand`

In [32]:
@macroexpand @assert 2+2 == 4

:(if 2 + 2 == 4
      nothing
  else
      Base.throw(Base.AssertionError("2 + 2 == 4"))
  end)

Each macro can define its own special syntax, and this is used extensively for
code introspection, serial performance improvements, and — perhaps most
importantly — parallelization perimitives!

## How is Julia fast?

By understanding the basics of how Julia _can_ be fast, you can get a better
sense for how to write fast Julia code.

Perhaps most importantly, Julia can reason about types. Recall: this is the definition of `findclosest2`:

```julia
function findclosest2(data, point)
    bestval = first(data)
    bestdist = abs(bestval - point)
    for elt in data
        dist = abs(elt - point)
        if dist < bestdist
            bestval = elt
            bestdist = dist
        end
    end
    return bestval
end
```

In [33]:
@code_typed optimize=false findclosest2(data, 0.5)

CodeInfo(
1 ─       (bestval = Main.first(data))::Float64
│   %2  = (bestval - point)::Float64
│         (bestdist = Main.abs(%2))::Float64
│   %4  = data::Array{Float64,1}
│         (@_6 = Base.iterate(%4))::Union{Nothing, Tuple{Float64,Int64}}
│   %6  = (@_6 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_6::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (elt = Core.getfield(%9, 1))::Float64
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Float64
│         (dist = Main.abs(%12))::Float64
│   %14 = (dist < bestdist)::Bool
└──       goto #4 if not %14
3 ─       (bestval = elt)::Float64
└──       (bestdist = dist)::Float64
4 ┄       (@_6 = Base.iterate(%4, %11))::Union{Nothing, Tuple{Float64,Int64}}
│   %19 = (@_6 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not %20
5 ─       goto #2
6 ┄       return bestval
) => Float64

In [34]:
typeof(data)

Array{Float64,1}

In [35]:
newdata = Real[data...]
typeof(newdata)

Array{Real,1}

In [36]:
@code_typed optimize=false findclosest2(newdata, 0.5)

CodeInfo(
1 ─       (bestval = Main.first(data))::Real
│   %2  = (bestval - point)::Any
│         (bestdist = Main.abs(%2))::Any
│   %4  = data::Array{Real,1}
│         (@_6 = Base.iterate(%4))::Union{Nothing, Tuple{Real,Int64}}
│   %6  = (@_6 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_6::Tuple{Real,Int64}::Tuple{Real,Int64}
│         (elt = Core.getfield(%9, 1))::Real
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Any
│         (dist = Main.abs(%12))::Any
│   %14 = (dist < bestdist)::Any
└──       goto #4 if not %14
3 ─       (bestval = elt)::Real
└──       (bestdist = dist)::Any
4 ┄       (@_6 = Base.iterate(%4, %11))::Union{Nothing, Tuple{Real,Int64}}
│   %19 = (@_6 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not %20
5 ─       goto #2
6 ┄       return bestval
) => Real

In [37]:
@benchmark findclosest2(newdata, 0.5)

BenchmarkTools.Trial: 
  memory estimate:  156.28 KiB
  allocs estimate:  10002
  --------------
  minimum time:     182.375 μs (0.00% GC)
  median time:      231.333 μs (0.00% GC)
  mean time:        236.051 μs (1.70% GC)
  maximum time:     1.919 ms (85.80% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [38]:
@code_warntype findclosest2(newdata, 0.5)

Variables
  #self#::Core.Compiler.Const(findclosest2, false)
  data::Array{Real,1}
  point::Float64
  bestval::Real
  bestdist::Any
  @_6::Union{Nothing, Tuple{Real,Int64}}
  elt::Real
  dist::Any

Body::Real
1 ─       (bestval = Main.first(data))
│   %2  = (bestval - point)::Any
│         (bestdist = Main.abs(%2))
│   %4  = data::Array{Real,1}
│         (@_6 = Base.iterate(%4))
│   %6  = (@_6 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_6::Tuple{Real,Int64}::Tuple{Real,Int64}
│         (elt = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Any
│         (dist = Main.abs(%12))
│   %14 = (dist < bestdist)::Any
└──       goto #4 if not %14
3 ─       (bestval = elt)
└──       (bestdist = dist)
4 ┄       (@_6 = Base.iterate(%4, %11))
│   %19 = (@_6 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not %20
5 ─       goto #2
6 ┄       return bestval


### Type stability

A function is called type-stable if Julia is able to infer what the output
type will be based purely on the types of the inputs.

Things that thwart type stability:
* Running things in global scope: create functions instead!
* Non-concretely typed containers
* Structs with abstractly-typed fields
* Non-constant globals (they might change!)
* Functions that change what they return based on the _values_:

#### More on macros

Each and every macro can define its own syntax. The `@benchmark` macro uses `$` in a special way.
The goal behind `@benchmark` is to evaluate the performance of a code snippet
as though it were written in a function. Use `$` to flag what will be an argument
or local variable in the function. Forgetting to use `$`s may result in faster
or slower timings than real-world performance.

In [39]:
x = 0.5 # non-constant global
@btime sin(x)
@btime sin($x)

  19.159 ns (1 allocation: 16 bytes)
  4.038 ns (0 allocations: 0 bytes)


0.479425538604203

In [40]:
@btime sin(0.5) # constant literal!
@btime sin($0.5)

  1.504 ns (0 allocations: 0 bytes)
  3.655 ns (0 allocations: 0 bytes)


0.479425538604203

### Specializations

Julia's reasoning about types is particularly important since it generates
specialized machine code specifically for the given arguments.

In [41]:
@code_llvm 1 + 2


;  @ int.jl:86 within `+'
define i64 @"julia_+_3407"(i64, i64) {
top:
  %2 = add i64 %1, %0
  ret i64 %2
}


This applies just the same to any functions we write — even the more complicated ones:

In [42]:
@code_llvm findclosest2(Float32[2.2,3.4,4.5],Float32(3.2))


;  @ In[29]:1 within `findclosest2'
define float @julia_findclosest2_3408(%jl_value_t* nonnull align 16 dereferenceable(40), float) {
top:
;  @ In[29]:2 within `findclosest2'
; ┌ @ abstractarray.jl:323 within `first'
; │┌ @ array.jl:809 within `getindex'
    %2 = bitcast %jl_value_t* %0 to %jl_array_t*
    %3 = getelementptr inbounds %jl_array_t, %jl_array_t* %2, i64 0, i32 1
    %4 = load i64, i64* %3, align 8
    %5 = icmp eq i64 %4, 0
    br i1 %5, label %oob, label %idxend

L48:                                              ; preds = %L48.preheader, %L48
    %6 = phi i64 [ %value_phi623, %L48 ], [ 1, %L48.preheader ]
    %value_phi4.25 = phi float [ %value_phi4., %L48 ], [ %20, %L48.preheader ]
    %value_phi3.value_phi524 = phi float [ %value_phi3.value_phi5, %L48 ], [ %17, %L48.preheader ]
    %value_phi623 = phi i64 [ %9, %L48 ], [ 2, %L48.preheader ]
; └└
;  @ In[29]:8 within `findclosest2'
; ┌ @ array.jl:785 within `iterate'
; │┌ @ array.jl:809 within `getindex'
    %7 = getel

This applies just the same to any functions we write — even the more complicated ones:

In [43]:
remove_comments(s) = join(filter(x->!startswith(x, ";"), split(s, "\n")), "\n")
sprint(code_llvm, findclosest2, Tuple{Vector{Float32}, Int}) |> remove_comments |> print


define float @julia_findclosest2_3409(%jl_value_t* nonnull align 16 dereferenceable(40), i64) {
top:
    %2 = bitcast %jl_value_t* %0 to %jl_array_t*
    %3 = getelementptr inbounds %jl_array_t, %jl_array_t* %2, i64 0, i32 1
    %4 = load i64, i64* %3, align 8
    %5 = icmp eq i64 %4, 0
    br i1 %5, label %oob, label %idxend

L50:                                              ; preds = %L50.preheader, %L50
    %6 = phi i64 [ %value_phi623, %L50 ], [ 1, %L50.preheader ]
    %value_phi4.25 = phi float [ %value_phi4., %L50 ], [ %21, %L50.preheader ]
    %value_phi3.value_phi524 = phi float [ %value_phi3.value_phi5, %L50 ], [ %17, %L50.preheader ]
    %value_phi623 = phi i64 [ %9, %L50 ], [ 2, %L50.preheader ]
    %7 = getelementptr inbounds float, float* %16, i64 %6
    %8 = load float, float* %7, align 4
    %9 = add i64 %value_phi623, 1
   %10 = fsub float %8, %18
   %11 = call float @llvm.fabs.f32(float %10)
   %12 = fcmp uge float %11, %value_phi4.25
   %value_phi3.value_phi5 = selec

## Modern hardware effects

There are lots of little performance quirks in modern computers; I'll just
cover two interesting ones here:

In [44]:
@benchmark findclosest2($data, $0.5)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.269 μs (0.00% GC)
  median time:      9.346 μs (0.00% GC)
  mean time:        9.561 μs (0.00% GC)
  maximum time:     28.398 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     5

In [45]:
sorteddata = sort(data)
@benchmark findclosest2($sorteddata, $0.5)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.148 μs (0.00% GC)
  median time:      9.377 μs (0.00% GC)
  mean time:        9.577 μs (0.00% GC)
  maximum time:     27.746 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     4

Unfortunately, this isn't demonstrable on a hardened cloud platform... because
it's a huge security risk!

* https://meltdownattack.com
* https://discourse.julialang.org/t/psa-microbenchmarks-remember-branch-history/17436

In [46]:
idxs = sortperm(data)
sortedview = @view data[idxs]
@benchmark findclosest2($sortedview, $0.5)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.258 μs (0.00% GC)
  median time:      8.359 μs (0.00% GC)
  mean time:        8.807 μs (0.00% GC)
  maximum time:     36.639 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     5

### Memory latencies

| System Event                   | Actual Latency | Scaled Latency |
| ------------------------------ | -------------- | -------------- |
| One CPU cycle                  |     0.4 ns     |     1 s        |
| Level 1 cache access           |     0.9 ns     |     2 s        |
| Level 2 cache access           |     2.8 ns     |     7 s        |
| Level 3 cache access           |      28 ns     |     1 min      |
| Main memory access (DDR DIMM)  |    ~100 ns     |     4 min      |
| Intel Optane memory access     |     <10 μs     |     7 hrs      |
| NVMe SSD I/O                   |     ~25 μs     |    17 hrs      |
| SSD I/O                        |  50–150 μs     | 1.5–4 days     |
| Rotational disk I/O            |    1–10 ms     |   1–9 months   |
| Internet call: SF to NYC       |      65 ms     |     5 years    |
| Internet call: SF to Hong Kong |     141 ms     |    11 years    |

 (from https://www.prowesscorp.com/computer-latency-at-a-human-scale/)

# Key Takeaways

* Measure, measure, measure!
* Get familiar with the [Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/)
* Don't be scared of `@code_typed`/`@code_warntype` and `@code_llvm`